In [1]:
import tensorflow as tf
import numpy as np

tf.constant()创建张量

In [2]:
#tensor类似Numpy的ndarray
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # 浮点数矩阵

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42) # 标量

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape #形状

TensorShape([2, 3])

In [5]:
t.dtype #数据类型

tf.float32

### 索引

In [6]:
t[:,1:] #取所有行，第1、2列

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis]
#tf.newaxis主要用途：增加一个维度

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [8]:
t[..., 1]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 5.], dtype=float32)>

In [9]:
t + 10 #加，等效于tf.add(t,10)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [10]:
tf.square(t) #取平方

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [11]:
t @ tf.transpose(t) 
#@运算符用于矩阵乘法，等效于调用tf.matmal()
#tf.transpose()将矩阵转置

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

### keras的底层API

In [12]:
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [13]:
from tensorflow import keras

K = keras.backend
#一般在需要编写可移植到其他Keras中实现的代码时需要使用

K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### Numpy和tf共同使用

numpy默认使用64位精度，数据类型为float64

tensorflow默认使用32位置精度，float32¶

In [14]:
#用numpy数组创建tensor
a = np.array([2., 4., 5.])
tf.constant(a)#应设置dtype=float32以便与直接创建的tensor相同

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [15]:
#将tensor转换为np的数组
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [16]:
#将numpy操作应用于tensor
np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [17]:
#将tensor操作应用于numpy数组
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [18]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

tf.cast(x，dtype):类型转换

In [19]:
try:
    tf.constant(2.0) + tf.constant(40)
    #浮点张量和整型张量不可相加
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]


In [20]:
try:
    tf.constant(2.0) + tf.constant(40., dtype=tf.float64)
    #32位浮点和64位浮点不可相加
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]


In [21]:
t2 = tf.constant(40., dtype=tf.float64)

#tf.constant(2.0)类型为float32
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

##### tf.Variable():变量

tf.Tensor值是不变的，因此不能使用它（常规向量）在神经网络中实现权重

keras中的add_weight可以实现手动创建变量

In [22]:
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

tf.assign(ref, value, validate_shape=True, use_locking=None, name=None)：将 value 赋值给 ref，修改变量

In [23]:
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [24]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [25]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [26]:
#不能直接修改变量
try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment


scatter_nd_update（）：修改变量的单个单元或切片

In [27]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]],
                    updates=[100., 200.])
#修改v[0][0]、v[1][2]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

In [28]:
#将v[1,:]和v[2,:]对调
sparse_delta = tf.IndexedSlices(values=[[1., 2., 3.], [4., 5., 6.]],
                                indices=[1, 0])
v.scatter_update(sparse_delta)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[4., 5., 6.],
       [1., 2., 3.]], dtype=float32)>

### 1. 字符串string：应用于自然语言处理

In [29]:
#用字符串创建tensor，string长度不是tensor.shape的一部分
tf.constant(b"hello world")

<tf.Tensor: shape=(), dtype=string, numpy=b'hello world'>

当使用Unicode字符串创建张量时，将其编码为utf8

In [30]:
#用Unicode字符串创建tensor，unicode字符串的长度时tensor.shape的一部分
tf.constant("人们")
#‘人’编码为0xe4 0xba 0xba，‘们’编码为0xe4 0xbb 0xac

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe4\xba\xba\xe4\xbb\xac'>

#### ord（）：把一个字符串表示的字符string→字符相对应的整数int32，适用于UNICODE字符。

In [31]:
#创建表示unicode字符串的tensor
u = tf.constant([ord(c) for c in "人们"])
u
#每个整数代表一个unicode码点（为每个符号唯一指定的编号）
#比如ord(‘a’)则返回整数97

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([20154, 20204])>

#### tf.strings.unicode_encode（）将unicode字符串tensor即int32→string字节字符串tensor

In [32]:
b = tf.strings.unicode_encode(u, "UTF-8")
b

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe4\xba\xba\xe4\xbb\xac'>

#### tf.strings.unicode_decode（）：string →unicode string tensor即int32

In [33]:
tf.strings.unicode_decode(b, "UTF-8")#反操作

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([20154, 20204])>

#### tf.strings.length()：计算字节字符串中的字节数，设置unit='UTF8_CHAR'时计算代码点数

In [34]:
tf.strings.length(b, unit="UTF8_CHAR")
#‘人们’的字节字符串有两个代码点

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [35]:
#操作包含多个字符串的tensor
p = tf.constant(["Café", "Coffee", "caffè", "咖啡"])
tf.strings.length(p, unit="UTF8_CHAR")#计算代码点数

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([4, 6, 5, 2])>

In [36]:
r = tf.strings.unicode_decode(p, "UTF8")#反编码，string →int32的unicode string tensor
r
print(r)

<tf.RaggedTensor [[67, 97, 102, 233], [67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232], [21654, 21857]]>


### 2. 不规则张量

不规则张量代表不同大小的数组列表，其切片可能具有不同长度的尺寸

第一个维度是规则的维度（统一维度），第二个维度是不规则的维度

In [37]:
#不规则张量r所有元素都是规则tensor
print(r[1])#第二个元素（数组）

tf.Tensor([ 67 111 102 102 101 101], shape=(6,), dtype=int32)


In [38]:
print(r[1:3])#第2、3个元素的切片

<tf.RaggedTensor [[67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232]]>


#### 创建和操作不规则tensor

tf.concat()：拼接数组

In [39]:
r2 = tf.ragged.constant([[65, 66], [], [67]])
print(tf.concat([r, r2], axis=0))#在第0维拼接

<tf.RaggedTensor [[67, 97, 102, 233], [67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232], [21654, 21857], [65, 66], [], [67]]>


In [40]:
r3 = tf.ragged.constant([[68, 69, 70], [71], [], [72, 73]])
print(tf.concat([r, r3], axis=1))

<tf.RaggedTensor [[67, 97, 102, 233, 68, 69, 70], [67, 111, 102, 102, 101, 101, 71], [99, 97, 102, 102, 232], [21654, 21857, 72, 73]]>


In [41]:
tf.strings.unicode_encode(r3, "UTF-8")#解码成unicode-int32的字符串

<tf.Tensor: shape=(4,), dtype=string, numpy=array([b'DEF', b'G', b'', b'HI'], dtype=object)>

In [42]:
r

<tf.RaggedTensor [[67, 97, 102, 233], [67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232], [21654, 21857]]>

to_tensor()：将不规则张量RaggedTensor转换为普通张量

In [43]:
r.to_tensor()

<tf.Tensor: shape=(4, 6), dtype=int32, numpy=
array([[   67,    97,   102,   233,     0,     0],
       [   67,   111,   102,   102,   101,   101],
       [   99,    97,   102,   102,   232,     0],
       [21654, 21857,     0,     0,     0,     0]])>

In [44]:
r.to_tensor(default_value=50)#设置填充值为50

<tf.Tensor: shape=(4, 6), dtype=int32, numpy=
array([[   67,    97,   102,   233,    50,    50],
       [   67,   111,   102,   102,   101,   101],
       [   99,    97,   102,   102,   232,    50],
       [21654, 21857,    50,    50,    50,    50]])>

### 3. 稀疏张量

tf.SparseTensor():构造一个稀疏矩阵类

In [45]:
s = tf.SparseTensor(indices=[[0, 1], [1, 0], [2, 3]],
                    values=[1., 2., 3.],
                    dense_shape=[3, 4])
print(s)

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


tf.sparse.to_dense():将稀疏表示形式转换为稠密张量.

In [46]:
tf.sparse.to_dense(s)
#将s[0][1]=1.，s[1][0]=2.,s[2][3]=3.,其余补位0，组成一个形状为[3,4]的矩阵

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 3.]], dtype=float32)>

In [47]:
s2 = s * 2.0
#稀疏张量可以与标量相乘

In [48]:
#稀疏向量不可以与标量相加
try:
    s3 = s + 1.
except TypeError as ex:
    print(ex)

unsupported operand type(s) for +: 'SparseTensor' and 'float'


In [49]:
print(s)

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


tf.sparse.sparse_dense_matmul( sp_a, b):将稀疏张量a 乘以密集矩阵b

In [50]:
s4 = tf.constant([[10., 20.], [30., 40.], [50., 60.], [70., 80.]])
tf.sparse.sparse_dense_matmul(s, s4)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 30.,  40.],
       [ 20.,  40.],
       [210., 240.]], dtype=float32)>

In [51]:
s5 = tf.SparseTensor(indices=[[0, 2], [0, 1]],
                     values=[1., 2.],
                     dense_shape=[3, 4])
##索引没有按照“读取顺序”列出
print(s5)

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]], shape=(2, 2), dtype=int64), values=tf.Tensor([1. 2.], shape=(2,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


In [52]:
try:
    tf.sparse.to_dense(s5)
    #索引必须以“读取顺序”列出（从左到右，从上到下）
    #不确定时使用tf.sparse.reorder（）将索引正确列出
except tf.errors.InvalidArgumentError as ex:
    print(ex)

indices[1] = [0,1] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SparseToDense]


In [53]:
s6 = tf.sparse.reorder(s5)
tf.sparse.to_dense(s6)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 2., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### 4. 张量数组

tf.TensorArray（）：张量的一个列表；
在包含循环的动态模型中很方便，可累加结果并计算统计信息

In [54]:
array = tf.TensorArray(dtype=tf.float32, size=3)
#所有元素形状必须与写入数组的第一个元素相同

array = array.write(0, tf.constant([1., 2.]))
array = array.write(1, tf.constant([3., 10.]))
array = array.write(2, tf.constant([5., 7.]))

stack()：将元素堆叠到一个常规张量中

In [55]:
array.stack()
#原张量列表

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 1.,  2.],
       [ 3., 10.],
       [ 5.,  7.]], dtype=float32)>

In [56]:
#读取一个元素会从数组中弹出该元素，并用相同形状的零张量替换它
array.read(1)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 3., 10.], dtype=float32)>

In [57]:
array.stack()
#读取一个元素之后的张量列表

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 2.],
       [0., 0.],
       [5., 7.]], dtype=float32)>

In [58]:
mean, variance = tf.nn.moments(array.stack(), axes=0)
mean

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 3.], dtype=float32)>

In [59]:
variance

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([4.6666665, 8.666667 ], dtype=float32)>

### 5. 集合

tf.sets.union（a，b）：计算 a 和 b 的最后一维元素的集合并集

In [60]:
set1 = tf.constant([[2, 3, 5, 7], [7, 9, 0, 0]])
set2 = tf.constant([[4, 5, 6], [9, 10, 0]])
tf.sparse.to_dense(tf.sets.union(set1, set2))#转换为稠密tensor，默认用0填充

<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 2,  3,  4,  5,  6,  7],
       [ 0,  7,  9, 10,  0,  0]])>

tf.sets.difference（a，b）：计算 a 和 b 的最后一维元素的集合差

In [61]:
tf.sparse.to_dense(tf.sets.difference(set1, set2))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[2, 3, 7],
       [7, 0, 0]])>

tf.sets.intersection（a，b）：计算 a 和 b 的最后一维元素的集合交集

In [62]:
tf.sparse.to_dense(tf.sets.intersection(set1, set2))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[5, 0],
       [0, 9]])>